In [ ]:
import time
import ROOT
import numpy as np
import pandas as pd
import root_pandas as rpd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
from root_pandas import read_root
from matplotlib import rc
from numpy import inf

from ROOT import TLatex

rc('text', usetex=True)

import matplotlib as mpl
rc('font', family='serif')
rc('text', usetex=True)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

#
mpl.rcParams.update({'font.size': 19})
#mpl.rcParams.update({'legend.fontsize': 18})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})
mpl.rcParams.update({'axes.labelsize': 18}) 
mpl.rcParams.update({'legend.frameon': False}) 

In [ ]:
# Define default plot styles
plot_style_0 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'linestyle': '--',
    'density': True
}

plot_style_1 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'density': True
}

plot_style_2 = {'alpha': 0.5, 'density': True}

In [ ]:

#load libraries 
import sys,pandas as pd, matplotlib , matplotlib.pyplot as plt, matplotlib.lines , numpy as np, math, pylab
#import ROOT
#from ROOT import TFile

import root_pandas
%matplotlib inline

#presets for plot style
from scipy.constants import golden
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (7, 7/golden),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

In [ ]:
def applyCut(inputDataframe, cut, text=None):
    dataframe = inputDataframe
    nbeforecut = dataframe.shape[0]
    cutDataframe = dataframe.query(cut)
    if text:
        print (text, cutDataframe.shape[0], ' fraction kept: %2.1f'%(100.0*float(cutDataframe.shape[0])/nbeforecut))
    return cutDataframe

In [ ]:
def applyCutsElectrons(df,isMC=False):
    temp = df
    temp = applyCut(temp, 'TargType==2')
    temp = applyCut(temp, 'y < 0.85', 'y< 0.85')

    return temp

In [ ]:
path = '/home/miguel/EG2_DATA/sebastian/'
data_electrons = root_pandas.read_root(path+'C.root', "ntuple_data_electrons")
data_hadron = root_pandas.read_root(path+'C.root', "ntuple_data")

In [ ]:
##select only solid target

In [ ]:
data_electrons = data_electrons.query('TargType==2')
data_hadron    = data_hadron.query('TargType==2')

## obtain MC

In [ ]:
path = '/home/miguel/EG2_DATA/andres/'
mc_electrons = root_pandas.read_root(path+'MC_C.root', "ntuple_e")
mc_hadron = root_pandas.read_root(path+'MC_C.root', "ntuple_sim")

In [ ]:
#keep only relevant variables in ntuple
#mc_hadron = mc_hadron[['Px','''P','mc_P','pid','ThetaLab','mc_ThetaLab','Zh','mc_Zh','PhiPQ','mc_PhiPQ','Mx2','mc_Mx2','Pt2','mc_Pt2']]
mc_hadron['PID'] = mc_hadron['pid']

In [ ]:
mc_hadron['pass_reco'] = np.where(mc_hadron['Pt2']>0, True, False)
mc_hadron['pass_true'] = np.where(mc_hadron['mc_Pt2']>0, True, False)


In [ ]:
## For the moment, just keep entries with valid generated pT2. 
## This is to remove 

In [ ]:
mc_hadron = mc_hadron.query('mc_Pt2>0')


In [ ]:
data_electrons.hist(figsize=(12,12))

In [ ]:
mc_electrons.hist(figsize=(12,12))

In [ ]:
print (data_electrons.shape[0])
print (mc_electrons.shape[0])

In [ ]:
data_hadron.keys()

In [ ]:
df[df['hasimage'] == False]

In [ ]:
mc_hadron[mc_hadron['pass_true']==True].hist(figsize=(12,12))
plt.show()


In [ ]:
mc_hadron.hist(figsize=(12,12),bins=100)
plt.show()

## pT vs z 

In [ ]:
fig, axs = plt.subplots(1, 5, sharey=True, sharex=True, figsize=(16,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data_hadron.query(query)['Pt2'],density=True,range=(0.0,1.80),bins=10,alpha=0.6,label='data')
    axs[j].hist(mc_hadron.query(query)['Pt2'],density=True,range=(0.0,1.80),bins=10,alpha=0.6,label='MC')

    axs[j].set_yscale('log')
    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]))
    axs[j].set_xlabel(r'$p_{T}^{2}$')

    print(query)
axs[0].legend()

## Phi PQ

In [ ]:
fig, axs = plt.subplots(1, 5, sharey=True, figsize=(16,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data_hadron.query(query)['PhiPQ'],density=True,bins=20,alpha=0.5)
    axs[j].hist(mc_hadron.query(query)['PhiPQ'],density=True,bins=20,alpha=0.5)

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]))
    axs[j].set_xlabel(r'$\phi_{PQ}$')

    #axs[j].set_yscale('log')

    print(query)

## ThetaLab

In [ ]:
fig, axs = plt.subplots(1, 5, sharey=True, sharex=True, figsize=(16,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data_hadron.query(query)['ThetaLab'],density=True,bins=20,alpha=0.5)
    axs[j].hist(mc_hadron.query(query)['ThetaLab'],density=True,bins=20,alpha=0.5)

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]))
    axs[j].set_xlabel(r'$\theta_{LAB}$')

    #axs[j].set_yscale('log')

    print(query)

## P momentum

In [ ]:
fig, axs = plt.subplots(1, 5, sharey=True, sharex=True, figsize=(16,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data_hadron.query(query)['P'],density=True,range=(0.0,3.5),bins=20,alpha=0.5)
    axs[j].hist(mc_hadron.query(query)['P'],density=True,range=(0.0,3.5),bins=20,alpha=0.5)

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]))
    axs[j].set_xlabel(r'$P$')

    #axs[j].set_yscale('log')

    print(query)

## Px

In [ ]:
fig, axs = plt.subplots(1, 5, sharey=True, sharex=True, figsize=(16,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data_hadron.query(query)['Px'],density=True,range=(-1.5,1.5),bins=40,alpha=0.5)
    axs[j].hist(mc_hadron.query(query)['Px'],density=True,range=(-1.5,1.5),bins=40,alpha=0.5)

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]))
    axs[j].set_xlabel(r'$P_{x}$')

    #axs[j].set_yscale('log')

    print(query)

In [ ]:
fig, axs = plt.subplots(1, 5, sharey=True, sharex=True, figsize=(16,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data_hadron.query(query)['Py'],density=True,range=(-1.5,1.5),bins=40,alpha=0.5)
    axs[j].hist(mc_hadron.query(query)['Py'],density=True,range=(-1.5,1.5),bins=40,alpha=0.5)

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]))
    axs[j].set_xlabel(r'$P_{y}$')

    #axs[j].set_yscale('log')

    print(query)

## Electron variables

In [ ]:
mc_electrons = mc_electrons.query('mc_Q2>0 and Q2>0')

In [ ]:
mc_electrons.keys()

In [ ]:
fig, axs = plt.subplots(2, 3, sharey=False, figsize=(16,8))
axs[0,0].hist(data_electrons['Xb'],density=True,alpha=0.6,range=(0.0,1.0),bins=20)
axs[0,0].hist(mc_electrons['Xb'],density=True,alpha=0.6,range=(0,1.0),bins=20)
axs[0,0].set_xlabel(r'$X_{B}$')

axs[0,1].hist(data_electrons['Q2'],density=True,range=(1.0,4.0),alpha=0.6)
axs[0,1].hist(mc_electrons['Q2'],density=True,range=(1.0,4.0),alpha=0.6)
axs[0,1].set_xlabel(r'$Q^{2}$')

axs[0,2].hist(data_electrons['Nu'],density=True,bins=25,range=(2.0,4.5),alpha=0.5)
axs[0,2].hist(mc_electrons['Nu'],density=True,bins=25,range=(2.0,4.5),alpha=0.5)
axs[0,2].set_xlabel(r'$\nu$')

axs[1,0].hist(data_electrons['P'],density=True,bins=25,alpha=0.5)
axs[1,0].hist(mc_electrons['P'],density=True,bins=25,alpha=0.5)
axs[1,0].set_xlabel(r'$P$')

axs[1,1].hist(data_electrons['Pz'],density=True,bins=25,alpha=0.5)
axs[1,1].hist(mc_electrons['Pz'],density=True,bins=25,alpha=0.5)
axs[1,1].set_xlabel(r'$P_{z}$')

axs[1,2].hist(data_electrons['ThetaLab'],density=True,bins=25,alpha=0.5)
axs[1,2].hist(mc_electrons['ThetaLab'],density=True,bins=25,alpha=0.5)
axs[1,2].set_xlabel(r'$\theta_{lab}$')

plt.show()

In [ ]:
from __future__ import absolute_import, division, print_function

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Patch

from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import Dense, Input
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
# Input multiple observables as tuple or list (i.e. theta0_G = (obs_1, obs_2, ...))
"""    Arguments:
        num_observables: number of observables to 
                            simultaneously unfold (integer)
                            
        iterations: number of iterations (integer)
        
        theta0_G: tuple or list of nominal 
                    generation-level observables as Numpy arrays
                    
        theta0_S: tuple or list of nominal 
                    simulation-level observables as Numpy arrays
                    
        theta_unknown_S: tuple or list of "natural" 
                            (unknown) simulation-level observables 
                            to be unfolded as Numpy arrays
        
        Returns:
        - A Numpy array of weights to reweight distributions in 
        theta0_G to the unfolded distribution of theta_unknown_S
        
        - The model used to calculate those weights
"""

# from NN (DCTR)
def reweight(events):
    f = model.predict(events, batch_size=10000)
    weights = f / (1. - f)
    return np.squeeze(np.nan_to_num(weights))


def multifold(num_observables, iterations, theta0_G, theta0_S,
              theta_unknown_S,theta_unknown_G):
    
    theta0 = np.stack([theta0_G, theta0_S], axis=1)
    labels0 = np.zeros(len(theta0))
    theta_unknown = np.stack([theta_unknown_S, theta_unknown_G], axis=1)
    labels_unknown = np.ones(len(theta_unknown))
    
    xvals_1 = np.concatenate((theta0_S, theta_unknown_S))
    yvals_1 = np.concatenate((labels0, labels_unknown))

    xvals_2 = np.concatenate((theta0_G, theta0_G))
    yvals_2 = np.concatenate((labels0, labels_unknown))

    weights = np.empty(shape=(iterations, 2, len(theta0)))
# shape = (iteration, step, event)

    inputs = Input((1, ))
    hidden_layer_1 = Dense(50, activation='relu')(inputs)
    hidden_layer_2 = Dense(50, activation='relu')(hidden_layer_1)
    hidden_layer_3 = Dense(50, activation='relu')(hidden_layer_2)
    outputs = Dense(1, activation='sigmoid')(hidden_layer_3)
    
    model = Model(inputs=inputs, outputs=outputs)

    earlystopping = EarlyStopping(patience=10,
                              verbose=1,
                              restore_best_weights=True)
   

    weights_pull = np.ones(len(theta0_S))
    weights_push = np.ones(len(theta0_S))
    
    for i in range(iterations):
        print("\nITERATION: {}\n".format(i + 1))

        # STEP 1: classify Sim. (which is reweighted by weights_push) to Data
        # weights reweighted Sim. --> Data
        print("STEP 1\n")

        weights_1 = np.concatenate((weights_push, np.ones(len(theta_unknown_S))))
        # actual weights for Sim., ones for Data (not MC weights)

        X_train_1, X_test_1, Y_train_1, Y_test_1, w_train_1, w_test_1 = train_test_split(
            xvals_1, yvals_1, weights_1)

        model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
        model.fit(X_train_1[X_train_1!=-10],
              Y_train_1[X_train_1!=-10],
              sample_weight=w_train_1[X_train_1!=-10],
              epochs=200,
              batch_size=10000,
              validation_data=(X_test_1[X_test_1!=-10], Y_test_1[X_test_1!=-10], w_test_1[X_test_1!=-10]),
              callbacks=[earlystopping],
              verbose=1)

        weights_pull = weights_push * reweight(theta0_S)
        weights_pull[theta0_S==-10] = 1. #these are events that don't pass reco; take the prior.
        weights[i, :1, :] = weights_pull

    # STEP 2: classify Gen. to reweighted Gen. (which is reweighted by weights_pull)
    # weights Gen. --> reweighted Gen.
        print("\nSTEP 2\n")

        weights_2 = np.concatenate((np.ones(len(theta0_G)), weights_pull))
    # ones for Gen. (not MC weights), actual weights for (reweighted) Gen.

        X_train_2, X_test_2, Y_train_2, Y_test_2, w_train_2, w_test_2 = train_test_split(
        xvals_2, yvals_2, weights_2)

        model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
        model.fit(X_train_2,
              Y_train_2,
              sample_weight=w_train_2,
              epochs=200,
              batch_size=10000,
              validation_data=(X_test_2, Y_test_2, w_test_2),
              callbacks=[earlystopping],
              verbose=1)

        weights_push = reweight(theta0_G)
        weights[i, 1:2, :] = weights_push
    
    
    return weights, model

In [ ]:
weights[-1, 1, :]
weights[-1, 0, :]

## Hadron

In [ ]:
data_hadron.keys()

In [ ]:
data = mc_hadron.sample(n=500000)
mc   = mc_hadron.sample(n=500000)

In [ ]:
print(len(mc))

In [ ]:
Px_0_G = mc['mc_Px']
Px_0_S =mc['Px']
Px_unknown_S = data['Px']
Px_unknown_G = data['mc_Px']

Px_0_G[mc['pass_true']==True] = -10
Px_0_S[mc['pass_reco']==True] = -10
Px_unknown_G[data['pass_true']==True] = -10
Px_unknown_S[data['pass_reco']==True] = -10


Py_0_G = mc['mc_Px']
Py_0_S =mc['Px']
Py_unknown_S = data['Px']
Py_unknown_G = data['mc_Px']

Py_0_G[mc['pass_true']==True] = -10
Py_0_S[mc['pass_reco']==True] = -10
Py_unknown_G[data['pass_true']==True] = -10
Py_unknown_S[data['pass_reco']==True] = -10

In [ ]:
K.clear_session()


In [ ]:
num_observables = 3

# simple / 100. standardization s.t. data is of order ~1

weights, _ = multifold(num_observables=num_observables,
                       iterations=1,
                       theta0_G=(Px_0_G, Py_0_G),
                       theta0_S=(Px_0_S, Py_0_S),
                       theta_unknown_S=(Px_unknown_S, Py_unknown_S),
                       theta_unknown_G=(Px_unknown_G, Py_unknown_G)
                                            
                      )

In [ ]:
for i in range(len(weights)):

    print("ITERATION: {}".format(i + 1))
    bins = np.linspace(1.0, 4.0, 40)
    
    fig, ax = plt.subplots(3,
                           3,
                           figsize=(16, 16),
                           constrained_layout=True)
    ax[0,0].set_xlabel('Q2')
    ax[0,0].set_ylabel('Events per bin (Normalized)')
    ax[0,0].hist(Q2_0_S, bins=bins, label=r'MC .', **plot_style_2)
    ax[0,0].hist(Q2_0_S,
               bins=bins,
               weights=weights[i, 0, :],
               label=r'MC . DCTR wgt.',
               **plot_style_1)

    ax[0,0].hist(Q2_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    ax[0,0].legend(frameon=False,loc='best')
   
    bins = np.linspace(2.0, 5.0, 40)

    
    ax[0,1].set_xlabel('Nu')
    ax[0,1].set_ylabel('Events per bin (Normalized)')
    ax[0,1].hist(Nu_0_S, bins=bins, label=r'MC (reco).', **plot_style_2)
    ax[0,1].hist(Nu_0_S,
               bins=bins,
               weights=weights[i, 0, :],
               label=r'MC (reco). DCTR wgt.',
               **plot_style_1)

    ax[0,1].hist(Nu_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    bins = np.linspace(-1.5, 1.5, 40)

    
    ax[0,2].set_xlabel('Px')
    ax[0,2].set_ylabel('Events per bin (Normalized)')
    ax[0,2].hist(Px_0_S, bins=bins, label=r'MC (reco).', **plot_style_2)
    ax[0,2].hist(Px_0_S,
               bins=bins,
               weights=weights[i, 0, :],
               label=r'MC (reco). DCTR wgt.',
               **plot_style_1)

    ax[0,2].hist(Px_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    bins = np.linspace(-1.5, 1.5, 40)

    
    ax[1,0].set_xlabel('Py')
    ax[1,0].set_ylabel('Events per bin (Normalized)')
    ax[1,0].hist(Py_0_S, bins=bins, label=r'MC (reco).', **plot_style_2)
    ax[1,0].hist(Py_0_S,
               bins=bins,
               weights=weights[i, 0, :],
               label=r'MC (reco). DCTR wgt.',
               **plot_style_1)

    ax[1,0].hist(Py_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    bins = np.linspace(-1.5, 3.5, 40)

    
    ax[1,1].set_xlabel('Pz')
    ax[1,1].set_ylabel('Events per bin (Normalized)')
    ax[1,1].hist(Pz_0_S, bins=bins, label=r'MC (reco).', **plot_style_2)
    ax[1,1].hist(Pz_0_S,
               bins=bins,
               weights=weights[i, 0, :],
               label=r'MC (reco). DCTR wgt.',
               **plot_style_1)

    ax[1,1].hist(Pz_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    
      
    bins = np.linspace(0, 1.5, 40)

    
    ax[1,2].set_xlabel('Pt2')
    ax[1,2].set_ylabel('Events per bin (Normalized)')
    ax[1,2].hist(Pt2_0_S, bins=bins, label=r'MC (reco).', **plot_style_2)
    ax[1,2].hist(Pt2_0_S,
               bins=bins,
               weights=weights[i, 0, :],
               label=r'MC (reco). DCTR wgt.',
               **plot_style_1)

    ax[1,2].hist(Pt2_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    
    
    bins = np.linspace(0, 1.5, 40)

    
    ax[2,0].set_xlabel('Zh')
    ax[2,0].set_ylabel('Events per bin (Normalized)')
    ax[2,0].hist(Zh_0_S, bins=bins, label=r'MC (reco).', **plot_style_2)
    ax[2,0].hist(Zh_0_S,
               bins=bins,
               weights=weights[i, 0, :],
               label=r'MC (reco). DCTR wgt.',
               **plot_style_1)

    ax[2,0].hist(Zh_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    
    bins = np.linspace(-1.5, 1.5, 40)

    
    ax[2,1].set_xlabel('Xf')
    ax[2,1].set_ylabel('Events per bin (Normalized)')
    ax[2,1].hist(Xf_0_S, bins=bins, label=r'MC (reco).', **plot_style_2)
    ax[2,1].hist(Xf_0_S,
               bins=bins,
               weights=weights[i, 0, :],
               label=r'MC (reco). DCTR wgt.',
               **plot_style_1)

    ax[2,1].hist(Xf_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    
    
    
    bins = np.linspace(-180.0, 180.0, 40)

    ax[2,2].set_xlabel('PhiPQ')
    ax[2,2].set_ylabel('Events per bin (Normalized)')
    ax[2,2].hist(PhiPQ_0_S, bins=bins, label=r'MC (reco).', **plot_style_2)
    ax[2,2].hist(PhiPQ_0_S,
               bins=bins,
               weights=weights[i, 0, :],
               label=r'MC (reco). DCTR wgt.',
               **plot_style_1)

    ax[2,2].hist(PhiPQ_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    
    
    


In [ ]:
plt.hist(data.query(query)['ThetaLab'],density=True,bins=np.linspace(0,1,100),alpha=0.5,label='data')
plt.hist(mc.query(query)['ThetaLab'],density=True,bins=np.linspace(0,1,100),alpha=0.5,label='mc')
plt.hist(mc.query(query)['ThetaLab'],bins=np.linspace(0,1,100),weights = mc.query(query)['weight'],**plot_style_1,label='mc weighted')


In [ ]:
## Differential 

In [ ]:
mc['weight'] = weights[-1, 0, :]

In [ ]:
fig, axs = plt.subplots(1, 5, sharey=True, sharex=True, figsize=(20,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(0, 140, 50)

    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data.query(query)['ThetaLab'],density=True,bins=binning,alpha=0.5,label='data')
    axs[j].hist(mc.query(query)['ThetaLab'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['ThetaLab'],bins=binning,weights = mc.query(query)['weight'],**plot_style_1,label='mc weighted')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$\theta_{Lab}$',fontsize=25)

#axs[j].set_yscale('log')

    print(query)
axs[0].legend(frameon=False,loc='best')


In [ ]:
fig, axs = plt.subplots(1, 5, sharey=True, sharex=True, figsize=(20,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(-180, 180, 25)

    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data.query(query)['PhiPQ'],density=True,bins=binning,alpha=0.5,label='data')
    axs[j].hist(mc.query(query)['PhiPQ'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['PhiPQ'],bins=binning,weights = mc.query(query)['weight'],**plot_style_1,label='mc weighted')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$\phi_{PQ}$',fontsize=25)

#axs[j].set_yscale('log')

    print(query)
axs[0].legend(frameon=False,loc='best')


In [ ]:
fig, axs = plt.subplots(1, 5, sharey=True, sharex=True, figsize=(24,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(0, 2.0, 20)

    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data.query(query)['Pt2'],density=True,bins=binning,alpha=0.5,label='data')
    axs[j].hist(mc.query(query)['Pt2'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['Pt2'],bins=binning,weights = mc.query(query)['weight'],**plot_style_1,label='mc weighted')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$P_{T}^{2}$',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
axs[0].set_yscale('log')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True,figsize=(24,4))


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(-1.8, 1.8, 25)

    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data.query(query)['Px'],density=True,bins=binning,alpha=0.5,label='data')
    axs[j].hist(mc.query(query)['Px'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['Px'],bins=binning,weights = mc.query(query)['weight'],**plot_style_1,label='mc w.')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$P_{x}$',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, figsize=(24,4))


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(-1.5, 1.5, 25)

    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data.query(query)['Py'],density=True,bins=binning,alpha=0.5,label='data')
    axs[j].hist(mc.query(query)['Py'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['Py'],bins=binning,weights = mc.query(query)['weight'],**plot_style_1,label='mc w')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$P_{y}$',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, figsize=(24,4))


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(0.0, 4.5, 50)

    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data.query(query)['P'],density=True,bins=binning,alpha=0.5,label='data')
    axs[j].hist(mc.query(query)['P'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['P'],bins=binning,weights = mc.query(query)['weight'],**plot_style_1,label='mc w')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$P$',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, figsize=(24,4))


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(-1.2,1.2 , 40)

    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data.query(query)['Xf'],density=True,bins=binning,alpha=0.5,label='data')
    axs[j].hist(mc.query(query)['Xf'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['Xf'],bins=binning,weights = mc.query(query)['weight'],**plot_style_1,label='mc w')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$X_{F}$',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(24,4))


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(-3.0,3.0 , 40)

    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data.query(query)['deltaZ'],density=True,bins=binning,alpha=0.5,label='data')
    axs[j].hist(mc.query(query)['deltaZ'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['deltaZ'],bins=binning,weights = mc.query(query)['weight'],**plot_style_1,label='mc w')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=25)
    axs[j].set_xlabel(r'$\Delta$ z',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')



In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(24,4))


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(-3.0,3.0 , 40)

    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data.query(query)['YC'],density=True,bins=binning,alpha=0.5,label='data')
    axs[j].hist(mc.query(query)['YC'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['YC'],bins=binning,weights = mc.query(query)['weight'],**plot_style_1,label='mc w')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=25)
    axs[j].set_xlabel('YC',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')

In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(24,4),gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(-4.0,7.0 , 22)

    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data.query(query)['Sector_pi'],density=True,bins=binning,alpha=0.5,label='data')
    axs[j].hist(mc.query(query)['Sector_pi'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['Sector_pi'],bins=binning,weights = mc.query(query)['weight'],**plot_style_1,label='mc w')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=25)
    axs[j].set_xlabel('Sector',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(24,4),gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(1.0,4.0 , 40)

    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data.query(query)['Q2'],density=True,bins=binning,alpha=0.5,label='data')
    axs[j].hist(mc.query(query)['Q2'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['Q2'],bins=binning,weights = mc.query(query)['weight'],**plot_style_1,label='mc w')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$Q^{2}$',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(24,4),gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(2.0,4.5 , 20)

    query = 'Zh>%2.2f and Zh<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(data.query(query)['Nu'],density=True,bins=binning,alpha=0.5,label='data')
    axs[j].hist(mc.query(query)['Nu'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['Nu'],bins=binning,weights = mc.query(query)['weight'],**plot_style_1,label='mc w')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$\nu$',fontsize=22)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')
